In [ ]:
#| default_exp camera_talk
#| test: false

In [ ]:
import asyncio
#from DataTalks.arena_agent import parking_camera_loop
from agents import AgentHooks
from DataTalks.fastmcp_tools import app
import datetime
import random
from DataTalks.arena_agent import parking_q, _sse 
import logging


In [ ]:
#| export
#| eval: false

@app.on_event("startup")
async def start_parking_worker():
    try:
        asyncio.create_task(parking_camera_loop(), name="parking-camera-loop") 
        logging.info("✅ Camera talks...")
    except Exception as exc:
        logging.error("❌ Could not run Camera Talks")  
        


In [ ]:
#| export
#| eval: false


async def parking_camera_loop():
    """
    Periodically simulates new free-slot counts from a camera
    and pushes them into the parking_q as SSE blocks.
    """
    while True:
        free_slots_number = random.randint(0, 120)
        stamp = datetime.datetime.now().strftime("%H:%M:%S")

        # Make the text look like an assistant bubble
        # (e.g., "role: assistant" or just user-facing text)
        message_html = f"""
        <div class="chat chat-start">
          <div class="chat-header text-xs text-gray-500">assistant</div>
          <div class="chat-bubble bg-sky-700 text-white">
            Camera says: {free_slots_number} free slots now (at {stamp})!
          </div>
        </div>
        """
        # Put an SSE 'message' event into the queue
        sse_message = _sse("message", message_html)
        parking_q.put_nowait(sse_message)

        await asyncio.sleep(60)  

In [ ]:
#| hide
#| eval: false
